In [54]:
from google.colab import auth
import gspread
from google.auth import default
import pandas
import random
from numpy import int64

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

yir_spreadsheet_name = 'Test YIR Spreadsheet'
reviewer_spreadsheet_name_format = 'Test YIR - {}'
owner_emails = ['nickreid@gmail.com']


def get_yir_spreadsheet():
  return gc.open(yir_spreadsheet_name)

def get_reviewer_spreadsheet_name(initials):
  return reviewer_spreadsheet_name_format.format(initials)

def get_or_create_spreadsheet(worksheet_name, verbose=False):
  try:
    worksheet = gc.open(worksheet_name)
  except gspread.exceptions.SpreadsheetNotFound:
    worksheet = gc.create(worksheet_name)
    for _email in owner_emails:
      worksheet.share(_email, perm_type='user', role='writer')
    if verbose:
      print("Created spreadsheet",worksheet_name)
  return worksheet

def write_A1_query(df, offset=1):
  cols = [
    'ABCDEFGHIJKLMNOPQURSTUVWXYZ'[index]
    for index, col in enumerate(df.columns)
  ]
  return 'A{}:{}{}'.format(
    offset,
    cols[-1],
    len(df)+offset
  )

def write_worksheet(df, spreadsheet, worksheet_name):
  df = remove_int64(df)
  worksheet = spreadsheet.worksheet(worksheet_name)
  worksheet.update(
      values = [
        df.columns.values.tolist()
      ] + [
        [
          value if not pandas.isnull(value) else ''
          for value in row
        ]
        for row in df.values.tolist()
      ],
      range_name = write_A1_query(df)
  )

def remove_int64(df):
  return pandas.DataFrame(
    [
        [
            value if type(value) is not int64 else int(value)
            for value in row
        ]
        for row in df.values
    ],
    index = df.index,
    columns = df.columns
  )

def parse_worksheet(ws):
  # Added to handle worksheets with only a header row and no values
  values = ws.get_all_values()
  headers = values.pop(0)
  return pandas.DataFrame(
      values,
      columns=headers
  )

In [87]:
def make_empty_reviewer_screened_articles_spreadsheet():
  return pandas.DataFrame(
      [],
      columns=['PMID','Include','First Author', 'Title', 'Journal']
  )

def get_or_create_reviewer_spreadsheet(initials, verbose=False):
  spreadsheet_name = get_reviewer_spreadsheet_name(initials)
  return get_or_create_spreadsheet(spreadsheet_name, verbose)


def get_or_create_reviewer_articles_to_screen_spreadsheet(initials, verbose=False):
  reviewer_spreadsheet = get_or_create_reviewer_spreadsheet(initials, verbose)
  try:
    return reviewer_spreadsheet.worksheet('Articles to screen')
  except:
    reviewer_spreadsheet.add_worksheet('Articles to screen', rows=100, cols=20)
    write_worksheet(
        make_empty_reviewer_screened_articles_spreadsheet(),
        reviewer_spreadsheet,
        'Articles to screen'
    )
    if verbose:
      print('Created articles to screen worksheet for', spreadsheet_name)
    return get_or_create_reviewer_articles_to_screen_spreadsheet(initials, verbose)

In [56]:
yir_spreadsheet = get_yir_spreadsheet()
articles = pandas.DataFrame(yir_spreadsheet.worksheet('Articles').get_all_records())
reviewers = pandas.DataFrame(yir_spreadsheet.worksheet('Reviewers').get_all_records())

In [66]:
## Update Screening Assignments
yir_spreadsheet = get_yir_spreadsheet()
screening_assignments = parse_worksheet(
  yir_spreadsheet.worksheet('Screening Assignments')
)

articles_to_add = []

reviewers_initials = reviewers['Initials'].tolist()
for _initials in reviewers_initials:
  reviewer_articles_to_screen = parse_worksheet(
    get_or_create_reviewer_articles_to_screen_spreadsheet(_initials)
  )
  for _, _article in reviewer_articles_to_screen.iterrows():
    recorded_article = screening_assignments.query('PMID=={} and Initials=="{}"'.format(_article['PMID'], _initials))
    if len(recorded_article) == 0:
      articles_to_add.append({
          'PMID': _article['PMID'],
          'Initials': _initials,
          'Include': _article['Include'],
          'First Author': _article['First Author'],
          'Title': _article['Title'],
          'Journal': _article['Journal']
      })
    elif len(recorded_article) == 1:
      _existing_index = recorded_article.index[0]
      screening_assignments.loc[_existing_index, 'Include'] = _article['Include']
    else:
      print('WTF Multiple Records',_initials, _article['PMID'])
updated_assignments = pandas.concat(
    [
    screening_assignments,
    pandas.DataFrame(articles_to_add)
    ]
)
updated_assignments.sort_values(['Initials', 'PMID'])
write_worksheet(updated_assignments, yir_spreadsheet, 'Screening Assignments')

In [67]:
## Update Screened Articles
yir_spreadsheet = get_yir_spreadsheet()
screening_assignments = parse_worksheet(yir_spreadsheet.worksheet('Screening Assignments'))
screened_articles = parse_worksheet(yir_spreadsheet.worksheet('Screened Articles'))
articles = parse_worksheet(yir_spreadsheet.worksheet('Articles'))

screened_articles_map = {}
article_ids = screening_assignments['PMID'].unique()
for _id in article_ids:
  assignments = screening_assignments.query('PMID=={}'.format(_id))
  ratings = {}
  complete_ratings = []
  for _, _assignment in assignments.iterrows():
    ratings[_assignment['Initials']] = _assignment['Include']
    cleaned_rating = _assignment['Include'].lower().strip()
    if cleaned_rating in ['yes', 'no', 'y', 'n']:
      complete_ratings.append(cleaned_rating)
  include = ''
  num_yes = len(
      [r for r in complete_ratings if r[0]=='y']
  )
  num_no = len(
      [r for r in complete_ratings if r[0]=='n']
  )
  if num_yes + num_no >= 1:
    if num_yes > num_no:
      include = 'Yes'
    if num_no > num_yes:
      include = 'No'
    if num_no == num_yes:
      include = 'Conflict'
  screened_articles_map[_id] = {
      'Assigned To': ', '.join(sorted(ratings.keys())),
      'Completed': len(complete_ratings),
      'Included': include
  }


articles_to_add = []
for _id in screened_articles_map.keys():
  existing_article = screened_articles.query('PMID=={}'.format(_id))
  if len(existing_article) == 0:
    articles_to_add.append({
        'PMID': _id,
        'Included': screened_articles_map[_id]['Included'],
        'Assigned To': screened_articles_map[_id]['Assigned To'],
        'Completed': screened_articles_map[_id]['Completed']
    })
  elif len(existing_article) == 1:
    _existing_index = existing_article.index[0]
    screened_articles.loc[_existing_index, 'Included'] = screened_articles_map[_id]['Included']
    screened_articles.loc[_existing_index, 'Assigned To'] = screened_articles_map[_id]['Assigned To']
    screened_articles.loc[_existing_index, 'Completed'] = screened_articles_map[_id]['Completed']
  else:
    print(_id, 'has more than 2 rows in screened articles WTF')

updated_articles = pandas.concat(
    [
      screened_articles,
      pandas.DataFrame(articles_to_add)
    ]
)

for _index, _article in updated_articles.iterrows():
  if '' in [_article['First Author'], _article['Title'], _article['Journal']]:
    existing_article = articles.query('PMID=={}'.format(_article['PMID'])).iloc[0]
    updated_articles.loc[_index, 'First Author'] = existing_article['First Author']
    updated_articles.loc[_index, 'Title'] = existing_article['Title']
    updated_articles.loc[_index, 'Journal'] = existing_article['Journal/Book']

updated_articles.sort_values(['PMID'])
write_worksheet(updated_articles, yir_spreadsheet, 'Screened Articles')

In [68]:
## Update reviewer Screened Count
yir_spreadsheet = get_yir_spreadsheet()
screening_assignments = parse_worksheet(yir_spreadsheet.worksheet('Screening Assignments'))
reviewers = parse_worksheet(yir_spreadsheet.worksheet('Reviewers'))

for _index, _reviewer in reviewers.iterrows():
  reviewers_screened_articles = screening_assignments[screening_assignments['Initials']==_reviewer['Initials']]
  complete_screens = 0
  for _include in reviewers_screened_articles['Include']:
    cleaned_rating = _include.lower().strip()
    if cleaned_rating in ['yes', 'no', 'y', 'n']:
      complete_screens += 1
  reviewers.at[_index, 'Articles Screened'] = complete_screens
write_worksheet(reviewers, yir_spreadsheet, 'Reviewers')

In [88]:
## Assign Articles
yir_spreadsheet = get_yir_spreadsheet()
screening_assignments = parse_worksheet(yir_spreadsheet.worksheet('Screening Assignments'))
articles = parse_worksheet(yir_spreadsheet.worksheet('Articles'))
reviewers = parse_worksheet(yir_spreadsheet.worksheet('Reviewers'))
screened_articles = parse_worksheet(yir_spreadsheet.worksheet('Screened Articles'))

priority_list = ['PriorityA', 'PriorityB']

def get_unassigned_article_ids(amount):
  if not amount:
    return []
  # Make list of unassigned articles
  unassigned_article_ids = []
  for _article_id in articles[articles['Priority'].isin(priority_list)]['PMID']:
    if sum(screening_assignments['PMID']==_article_id) < 2:
      unassigned_article_ids.append(_article_id)
  if len(unassigned_article_ids) < amount:
    return []
  return random.sample(unassigned_article_ids, amount)

def get_assigned_articles_from_inactive_reviewers(initials, amount):
  if not amount:
    return []
  inactive_reviewers = reviewers[reviewers['Articles Screened']==0]['Initials']
  inactive_reviewer_article_ids = screening_assignments[screening_assignments['Initials'].isin(inactive_reviewers)]['PMID'].unique()
  screening_assignments_for_active_reviewers = screening_assignments[
      screening_assignments['Initials'].isin(inactive_reviewers)
  ]

  unassigned_article_ids = []
  for _article_id in inactive_reviewer_article_ids:
    if sum(screening_assignments_for_active_reviewers['PMID']==_article_id) >= 2:
      continue
    if initials in screening_assignments_for_active_reviewers.query('PMID=={}'.format(_article_id))['Initials'].to_list():
      continue
    unassigned_article_ids.append(_article_id)
  if len(unassigned_article_ids)==0:
    return []
  if len(unassigned_article_ids) < amount:
    amount = len(unassigned_article_ids)
  return random.sample(unassigned_article_ids, amount)

def get_articles_that_need_additional_review(initials, amount):
  screeners_assignments = screening_assignments.query('Initials=="{}"'.format(initials))
  needs_another_review = screened_articles[
      (screened_articles['Completed'] == 1)
      & ~(screened_articles['PMID'].isin(screeners_assignments['PMID']))
  ]
  assignable_articles = []
  for _, article in needs_another_review.iterrows():
    if len(screening_assignments.query('PMID=={}'.format(article['PMID'])))<3:
      assignable_articles.append(article['PMID'])
  if len(assignable_articles):
    if amount > len(assignable_articles):
      amount = len(assignable_articles)
    return random.sample(assignable_articles, amount)
  else:
    return []


def get_articles_to_tie_break(initials, amount):
  screeners_assignments = screening_assignments.query('Initials=="{}"'.format(initials))
  conflicts = screened_articles[
      (screened_articles['Included'] == 'Conflict')
      & ~(screened_articles['PMID'].isin(screeners_assignments['PMID']))
  ]
  assignable_conflicts = []
  for _, article in conflicts.iterrows():
    if len(screening_assignments.query('PMID=={}'.format(article['PMID'])))<3:
      assignable_conflicts.append(article['PMID'])
  if len(assignable_conflicts):
    if amount > len(assignable_conflicts):
      amount = len(assignable_conflicts)
    return random.sample(assignable_conflicts, amount)
  else:
    return []


article_reviews = []
for _, reviewer in reviewers.iterrows():
  try:
    reviewer_articles_to_screen = int(reviewer['Articles to screen'])
  except:
    print(reviewer['Initials'], 'no articles to screen')
  reviewer_articles_assigned = len(
      screening_assignments.query('Initials=="{}"'.format(reviewer['Initials']))
  )
  if reviewer_articles_assigned < reviewer_articles_to_screen:
    articles_to_assign = reviewer_articles_to_screen - reviewer_articles_assigned
    print("{} needs {} articles".format(reviewer['Initials'], articles_to_assign))
    unassigned_article_ids = get_articles_to_tie_break(reviewer['Initials'], articles_to_assign)
    if len(unassigned_article_ids):
      print('Assigned {} articles with conflicts to {}'.format(
          len(unassigned_article_ids),
          reviewer['Initials'])
      )
      articles_to_assign = articles_to_assign - len(unassigned_article_ids)
    if articles_to_assign:
      articles_set = get_assigned_articles_from_inactive_reviewers(reviewer['Initials'], articles_to_assign)
      print('Assigned {} articles from inactive reviewers to {}'.format(
          len(articles_set),
          reviewer['Initials']
      ))
      unassigned_article_ids = unassigned_article_ids + articles_set
      articles_to_assign = articles_to_assign - len(articles_set)
    if articles_to_assign:
      unassigned_article_ids = unassigned_article_ids + get_unassigned_article_ids(articles_to_assign)
      print('Assigning {} articles to {} ({} needed)'.format(len(unassigned_article_ids), reviewer['Initials'], articles_to_assign))

    assignments_to_add = []
    for _id in unassigned_article_ids:
      article = articles.query('PMID=="{}"'.format(int(_id))).iloc[0]
      assignments_to_add.append(
          {
            'Initials': reviewer['Initials'],
            'PMID': article['PMID'],
            'Include':"",
            'First Author': article['First Author'],
            'Title': article['Title'],
            'Journal': article['Journal/Book'],
            'PubMed Link': article['PubMed Link']
          }
      )
    screening_assignments = pandas.concat([
        screening_assignments,
        pandas.DataFrame(assignments_to_add)
    ])

screening_assignments = screening_assignments.sort_values(['Initials', 'PMID'])
write_worksheet(screening_assignments, yir_spreadsheet, 'Screening Assignments')


# Write screening assignments to individual spreadsheets if missing
for _initials in screening_assignments['Initials'].unique():
  existing_screening_articles = parse_worksheet(
      get_or_create_reviewer_articles_to_screen_spreadsheet(_initials)
  )
  articles_to_add = []
  for _, _article in screening_assignments.query('Initials=="{}"'.format(_initials)).iterrows():
    if 'PMID' not in existing_screening_articles or _article['PMID'] not in existing_screening_articles['PMID'].tolist():
      articles_to_add.append(_article)
  if articles_to_add:
    reviewer_screening_articles = pandas.concat(
        [
            existing_screening_articles,
            pandas.DataFrame(articles_to_add)[[col for col in screening_assignments.columns if col != 'Initials']]
        ]
    )
    write_worksheet(
        reviewer_screening_articles,
        get_or_create_reviewer_spreadsheet(_initials),
        'Articles to screen'
        )

In [82]:
articles.query('PMID=="{}"'.format(_id))

,PMID,PubMed Link,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI,Priority
1122,40840463,https://pubmed.ncbi.nlm.nih.gov/40840463/,Multimodal Integration in Health Care: Develop...,"Hao Y, Cheng C, Li J, Li H, Di X, Zeng X, Jin ...",J Med Internet Res. 2025 Aug 21;27:e76557. doi...,Hao Y,J Med Internet Res,2025,21/08/2025,PMC12370271,,10.2196/76557,PriorityB
